 # СберАвтоподписка. Episode II: Pазработка модели, предсказывающей целевое событие  

---

**Цель проекта**: разработать модель предсказания совершения одного из целевых действий ("Заказать звонок", "Оставить заявку") для сессий по введенным атрибутам типа utm_*, device_*, geo_* и упаковать модель в сервис.



**Целевая метрика**: ориентировочное значение roc-auc > 0.65 — факт совершения
пользователем целевого действия.

**Формат вывода ответа** - 0/1

**Скорость ответа сервиса** - не более 3 секунд

**Сервис** - это должен быть (минимум) - py-скрипт с инструкцией по запуску, (максимум) - localhost web app.

---

Так как время от времени я периодически читаю разные ТГ-каналы про аналитику, написание кодов и упрощение роботы, да и в Skillbox дают так же, как в институте только основы (около 20% материала, остальные 80% обучающийся должен искать самостоятельно на просторах интернета или в библиотеках) в одной статье я наткнулся на библиотеку для машинного обучения Feature-engine, а так же устанавливаю библиотеки colorama, bayesian-optimization.




### Импорт библиотек

#### Перед импортом библиотек предварительно устанавливается:
* `pip install bayesian-optimization` согласно [документации](https://github.com/fmfn/BayesianOptimization)
* `pip install feature-engine`
* `pip install bayesian-optimization`
* `pip install colorama`

In [1]:
pip install bayesian-optimization

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install feature-engine

In [3]:
pip install colorama

Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
from datetime import datetime
import warnings
from pathlib import Path
from typing import Union
from functools import partial

import numpy as np
import dill
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random


# препроцессинг и метрики
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.model_selection import (
    train_test_split, GridSearchCV, StratifiedKFold)
from sklearn.metrics import (
    roc_auc_score, accuracy_score, confusion_matrix, precision_score, 
    recall_score, f1_score, make_scorer, roc_curve)
from sklearn.pipeline import Pipeline
from feature_engine.encoding import RareLabelEncoder, OneHotEncoder
from feature_engine.wrappers import SklearnTransformerWrapper
from feature_engine.outliers import Winsorizer
from feature_engine.selection import (
    DropDuplicateFeatures, DropConstantFeatures, 
    DropCorrelatedFeatures, DropFeatures)
from feature_engine.transformation import YeoJohnsonTransformer
from bayes_opt import BayesianOptimization

# модели
from sklearn.base import BaseEstimator
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, HistGradientBoostingClassifier)

### Создание констант

In [5]:
# необходимо указать путь к папкам с данными и моделями

data_folder = Path('C:/Users/user/PycharmProjects/SBERautoScription/', 'data')
models_folder = Path('C:/Users/user/PycharmProjects/SBERautoScription/', 'models')

sessions_filename = 'C:/Users/user/PycharmProjects/SBERautoScription/data/ga_sessions.csv'
hits_filename = 'C:/Users/user/PycharmProjects/SBERautoScription/data/ga_hits.csv'

In [6]:
TEST_SIZE = 200_000
RANDOM_SEED = 0

### Настройка ноутбука

In [7]:
pd.set_option('display.max_columns', 100)
warnings.filterwarnings('ignore')

### Загрузка данных

Для файла `ga_hits.csv` нужно загрузить только колонки 'session_id' и 'event_action', так как остальные не используются.

In [8]:
sessions = pd.read_csv(data_folder / sessions_filename)
hits = pd.read_csv(data_folder / hits_filename, 
                   usecols=['session_id', 'event_action'])

In [9]:
sessions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860042 entries, 0 to 1860041
Data columns (total 18 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   session_id                object
 1   client_id                 object
 2   visit_date                object
 3   visit_time                object
 4   visit_number              int64 
 5   utm_source                object
 6   utm_medium                object
 7   utm_campaign              object
 8   utm_adcontent             object
 9   utm_keyword               object
 10  device_category           object
 11  device_os                 object
 12  device_brand              object
 13  device_model              object
 14  device_screen_resolution  object
 15  device_browser            object
 16  geo_country               object
 17  geo_city                  object
dtypes: int64(1), object(17)
memory usage: 1.8 GB


In [10]:
hits.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15726470 entries, 0 to 15726469
Data columns (total 2 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   session_id    object
 1   event_action  object
dtypes: object(2)
memory usage: 2.5 GB


## Подготовка данных

### Целевая переменная  

Целевая переменная считается положительной, если для сессии из `sessions` есть хотя бы одно целевое событие в `hits`. 

In [11]:
target_events = [
    'sub_car_request_submit_click', 'sub_car_claim_submit_click', 
    'sub_callback_submit_click', 'sub_call_number_click', 'sub_car_claim_click', 
    'sub_submit_success', 'sub_open_dialog_click']

In [12]:
hits['target'] = hits['event_action'].isin(target_events) # заполняет столбец hits['target'] 'True', если hits['event_action'] содержит целевую
is_target_event = hits.groupby('session_id')['target'].any().astype(int) # target группирует по id (session_id) и по столбцу 'target', то есть все одинаковые session_id собираются

In [13]:
hits['target']

0           False
1           False
2           False
3           False
4           False
            ...  
15726465    False
15726466    False
15726467    False
15726468    False
15726469    False
Name: target, Length: 15726470, dtype: bool

In [14]:
is_target_event

session_id
1000009318903347362.1632663668.1632663668    0
1000010177899156286.1635013443.1635013443    0
1000013386240115915.1635402956.1635402956    0
1000017303238376207.1623489300.1623489300    0
1000020580299877109.1624943350.1624943350    0
                                            ..
999960188766601545.1626816843.1626816843     0
99996598443387715.1626811203.1626811203      0
999966717128502952.1638428330.1638428330     0
999988617151873171.1623556243.1623556243     0
999989480451054428.1634311006.1634311006     0
Name: target, Length: 1734610, dtype: int32

In [15]:
target = pd.Series(is_target_event, index=sessions['session_id']).fillna(0.0) # Заполним пропуски 0,0
target.value_counts(dropna=False, normalize=True) # посчитаем значения, так и есть - целевых значений  2,7%

target
0.0    0.97295
1.0    0.02705
Name: proportion, dtype: float64

In [16]:
target

session_id
9055434745589932991.1637753792.1637753792    0.0
905544597018549464.1636867290.1636867290     0.0
9055446045651783499.1640648526.1640648526    0.0
9055447046360770272.1622255328.1622255328    0.0
9055447046360770272.1622255345.1622255345    0.0
                                            ... 
9055415581448263752.1640159305.1640159305    0.0
9055421130527858185.1622007305.1622007305    0.0
9055422955903931195.1636979515.1636979515    0.0
905543020766873816.1638189404.1638189404     0.0
9055430416266113553.1640968742.1640968742    0.0
Name: target, Length: 1860042, dtype: float64

In [17]:
del hits # удалим датафрейм за ненадобностью чтобы не занимал много памяти

### Заполнение пропусков  

Пропуски в колонке `device_screen_resolution` заполняем самым частым значением.  
Все остальные пропуски в колонках заполняем значением '(nan)'.

In [18]:
def fill_missings(data: pd.DataFrame) -> pd.DataFrame:
    """Заполняет пропущенные значения:
    * самым частым значением для `device_screen_resolution`;
    * значением '(nan)' во всех остальных случаях.
    """

    data = data.copy()

    if 'device_screen_resolution' in data.columns:
        # '414x896' - самое частое значение в 'device_screen_resolution'
        # согласно предварительному анализу данных
        data['device_screen_resolution'] = \
            data['device_screen_resolution'].replace(missing_values, '414x896')
    
    return data.fillna('(nan)')

### Генерация признаков

Создаётся множество дополнительных переменных: день недели и день месяца, является ли день выходным, час и минута посещения, ночью ли посещение, ширина, высота, площадь и соотношение экрана.

В том числе с дополнительными данными: является ли день празничным, является ли трафик органическим, из социальных ли сетей он, находится ли пользователь в большом городе или в московской области, расстояние до Москвы как численно, так и в виде категорий.

In [19]:
def distance_category(distance: float) -> str:
    """Возвращает категорию расстояния до Москвы."""

    if distance == -1: return 'no distance'
    elif distance == 0: return 'moscow'
    elif distance < 100: return '< 100 km'
    elif distance < 500: return '100-500 km'
    elif distance < 1000: return '500-1000 km'
    elif distance < 3000: return '1000-3000 km'
    else: return '>= 3000 km'

In [20]:
def create_features(data: pd.DataFrame) -> pd.DataFrame:
    """Создаёт новые признаки из существующих."""

    data = data.copy()
    
    # visit_date признаки 
    if 'visit_date' in data.columns:
        data['visit_date'] = data['visit_date'].astype('datetime64[ns]')
        data['visit_date_added_holiday'] = \
            data['visit_date'].isin(russian_holidays)
        # числовые признаки сделаем строго положительными 
        # для лучшей обработки на шаге с YeoJohnsonTransformer
        data['visit_date_weekday'] = data['visit_date'].dt.weekday + 1
        data['visit_date_weekend'] = data['visit_date'].dt.weekday > 4
        data['visit_date_day'] = data['visit_date'].dt.day + 1

    # visit_time признаки
    if 'visit_time' in data.columns:
        data['visit_time'] = data['visit_time'].astype('datetime64[ns]')
        data['visit_time_hour'] = data['visit_time'].dt.hour + 1
        data['visit_time_minute'] = data['visit_time'].dt.minute + 1
        data['visit_time_night'] = data['visit_time'].dt.hour < 9

    # utm_* признаки
    if 'utm_medium' in data.columns:
        data['utm_medium_added_is_organic'] = \
            data['utm_medium'].isin(organic_mediums)
    if 'utm_source' in data.columns: 
        data['utm_source_added_is_social'] = \
            data['utm_source'].isin(social_media_sources)
    
    # device_screen признаки
    if 'device_screen_resolution' in data.columns:
        name = 'device_screen_resolution'
        data[[name + '_width', name + '_height']] = \
            data[name].str.split('x', expand=True).astype(float)
        data[name + '_area'] = data[name + '_width'] * data[name + '_height']
        data[name + '_ratio'] = data[name + '_width'] / data[name + '_height']
        data[name + '_ratio_greater_1'] = data[name + '_ratio'] > 1

    # geo_city признаки 
    if 'geo_city' in data.columns:
        data['geo_city_added_is_moscow_region'] = \
            data['geo_city'].isin(moscow_region_cities)
        data['geo_city_added_is_big'] = data['geo_city'].isin(big_cities)
        data['geo_city_is_big_or_in_moscow_region'] = \
            data['geo_city_added_is_moscow_region'] \
            | data['geo_city_added_is_big']
        data['geo_city_added_distance_from_moscow'] = \
            data['geo_city'].apply(get_distance_from_moscow)
        data['geo_city_added_distance_from_moscow_category'] = \
            data['geo_city_added_distance_from_moscow'].apply(distance_category)

    return data

### Дополнительно

In [21]:
def set_index(data: pd.DataFrame, column: str = 'session_id') -> pd.DataFrame:
    """Устанавливает в качестве индекса датафрейма колонку `column`."""
    
    data = data.copy()

    if column in data.columns:
        data = data.set_index(column)
    
    return data

In [22]:
def converse_types(data: pd.DataFrame) -> pd.DataFrame:
    """Приводит типы переменных к float. В первую очередь 
    необходимо для преобразования bool значений.
    """

    return data.astype(float)

### Собираем пайплайн  

Пайплайн по подготовке данных состоит из 4 частей:  
1. Создание дополнительных признаков
2. Преобразование численных переменных
3. Преобразование категориальных переменных
4. Удаление лишних признаков

In [23]:
preprocessor = Pipeline(steps=[

    # Создание дополнительных признаков и
    # Приведение датафрейма к удобному виду 
    ('indexer', FunctionTransformer(set_index)), 
    ('imputer', FunctionTransformer(fill_missings)), 
    ('engineer', FunctionTransformer(create_features)), 
    ('dropper', DropFeatures(['client_id', 'visit_date', 'visit_time', 
                              'device_screen_resolution'])), 

    # Преобразования численных переменных
    ('normalization', YeoJohnsonTransformer()), 
    ('outlier_remover', Winsorizer()), 
    ('scaler', SklearnTransformerWrapper(StandardScaler())), 

    # Преобразования категориальных признаков
    ('rare_encoder', RareLabelEncoder(tol=0.05, replace_with='rare')), 
    ('onehot_encoder', OneHotEncoder(drop_last_binary=True)), 
    ('bool_converter', FunctionTransformer(converse_types)), 

    # Удаление дубликатов и коррелируемых признаков
    ('constant_dropper', DropConstantFeatures(tol=0.99)), 
    ('duplicated_dropper', DropDuplicateFeatures()), 
    ('correlated_dropper', DropCorrelatedFeatures(threshold=0.8)), 

])

In [24]:
preprocessor

Pipeline(steps=[('indexer',
                 FunctionTransformer(func=<function set_index at 0x000001901803FCE0>)),
                ('imputer',
                 FunctionTransformer(func=<function fill_missings at 0x000001901803F1A0>)),
                ('engineer',
                 FunctionTransformer(func=<function create_features at 0x000001901803F420>)),
                ('dropper',
                 DropFeatures(features_to_drop=['client_id', 'visit_date',
                                                'visit_time',
                                                'device_s...
                 SklearnTransformerWrapper(transformer=StandardScaler())),
                ('rare_encoder', RareLabelEncoder(replace_with='rare')),
                ('onehot_encoder', OneHotEncoder(drop_last_binary=True)),
                ('bool_converter',
                 FunctionTransformer(func=<function converse_types at 0x0000019058198220>)),
                ('constant_dropper', DropConstantFeatures(tol=0.99)),
                ('duplicated_dropper', DropDuplicateFeatures()),
                ('correlated_dropper', DropCorrelatedFeatures())])

## Моделирование

### Разделение данных  

Разделим данные на тренировочную, валидационную и тестовую выборки.  


In [25]:
X, X_test, y, y_test = train_test_split(
    sessions, target, test_size=TEST_SIZE, 
    stratify=target, random_state=RANDOM_SEED)

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=TEST_SIZE, 
    stratify=y, random_state=RANDOM_SEED)

print(f'train shapes: {X_train.shape} {y_train.shape}')
print(f'valid shapes: {X_valid.shape} {y_valid.shape}')
print(f'test  shapes: {X_test.shape} {y_test.shape}')

train shapes: (1460042, 18) (1460042,)
valid shapes: (200000, 18) (200000,)
test  shapes: (200000, 18) (200000,)


### Препроцессинг данных  

Преобразуем тренировочные и валидационные данные с помощью построенного конвейера.

In [26]:
X_train

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
1056663,547332070992313735.1638727047.1638727047,127435678.163873,2021-12-05,20:57:27,1,ZpYIoDJMcFzVoPFsHGJL,banner,TmThBvoCcwkCZZUWACYq,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,428x926,Safari,Russia,Novotroitsk
1409550,7049980726847853060.1629809978.1629809978,1641451550.162934,2021-08-24,15:00:00,5,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,PkybGvWbaqORmxjNunqZ,NaN,mobile,NaN,Xiaomi,NaN,393x851,Chrome,Russia,Balashikha
739873,4064566220882386809.1639930788.1639930788,946355569.163992,2021-12-19,19:19:48,2,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,NaN,339x753,Chrome,Russia,Almetyevsk
62902,1033849280328168809.1631647080.1631647080,240711793.163165,2021-09-14,22:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,414x896,Safari,Russia,Saint Petersburg
1297234,6544287746962620171.1638268683.1638268683,1523710728.163827,2021-11-30,13:38:03,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,NaN,393x851,Chrome,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1793000,8755082352522037809.1633552946.1633552946,2038451459.163355,2021-10-06,23:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Samsung,NaN,360x800,Chrome,Russia,Moscow
207261,1682284231298311525.1640424805.1640424805,391687320.1640424805,2021-12-25,12:33:25,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,375x812,Safari,Russia,Saint Petersburg
93412,1170303784354950884.1640345317.1640345317,272482583.164035,2021-12-24,14:28:37,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Windows,NaN,NaN,1920x1080,Chrome,Russia,Moscow
1568591,7753976912353269271.1634047522.1634047522,1805363435.163405,2021-10-12,17:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Xiaomi,NaN,393x851,Chrome,Russia,Novosibirsk


In [27]:
missing_values = [float('nan'), '(none)', '(not set)', '0x0']

In [28]:
russian_holidays = [
    '2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04', '2021-01-05', 
    '2021-01-06', '2021-01-07', '2021-01-08', '2021-02-22', '2021-02-23', 
    '2021-03-08', '2021-05-01', '2021-05-03', '2021-05-09', '2021-05-10', 
    '2021-06-12', '2021-06-14', '2021-11-04', '2021-11-05', '2021-12-31', 
    '2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05', 
    '2022-01-06', '2022-01-07', '2022-01-08', '2022-02-23', '2022-03-08', 
    '2022-05-01', '2022-05-02', '2022-05-09', '2022-06-12', '2022-06-13', 
    '2022-11-04', '2022-12-31']

In [29]:
organic_mediums = ['organic', 'referral', '(none)']

In [30]:
social_media_sources = ['QxAxdyPLuQMEcrdZWdWb', 'MvfHsxITijuriZxsqZqt', 
                        'ISrKoXQCxqqYvAZICvjs', 'IZEXUFLARCUMynmHNBGo', 
                        'PlbkrSYoHuZBWfYjYnfw', 'gVRrcxiDQubJiljoTbGm']

In [31]:
moscow_region_cities = [
    'Aprelevka', 'Balashikha', 'Chekhov', 'Chernogolovka', 'Dedovsk', 
    'Dmitrov', 'Dolgoprudny', 'Domodedovo', 'Dubna', 'Dzerzhinsky', 
    'Elektrogorsk', 'Elektrostal', 'Elektrougli', 'Fryazino', 'Golitsyno', 
    'Istra', 'Ivanteyevka', 'Kalininets', 'Kashira', 'Khimki', 'Khotkovo', 
    'Klimovsk', 'Klin', 'Kolomna', 'Korolyov', 'Kotelniki', 'Krasnoarmeysk', 
    'Krasnogorsk', 'Krasnoznamensk', 'Kubinka', 'Kurovskoye', 
    'Likino-Dulyovo', 'Lobnya', 'Losino-Petrovsky', 'Lukhovitsy', 
    'Lytkarino', 'Lyubertsy', 'Mozhaysk', 'Mytishchi', 'Naro-Fominsk', 
    'Noginsk', 'Odintsovo', 'Orekhovo-Zuyevo', 'Pavlovsky Posad', 'Podolsk', 
    'Protvino', 'Pushchino', 'Pushkino', 'Ramenskoye', 'Reutov', 'Ruza', 
    'Sergiyev Posad', 'Serpukhov', 'Shatura', 'Shchyolkovo', 
    'Solnechnogorsk', 'Staraya Kupavna', 'Stupino', 'Vidnoye', 
    'Volokolamsk', 'Voskresensk', 'Yakhroma', 'Yegoryevsk', 'Zvenigorod']

In [32]:
big_cities = ['Moscow', 'Saint Petersburg', 'Novosibirsk', 'Yekaterinburg', 
              'Kazan', 'Nizhny Novgorod', 'Chelyabinsk', 'Samara', 'Omsk', 
              'Rostov-on-Don', 'Ufa', 'Krasnoyarsk', 'Voronezh', 'Perm', 
              'Volgograd', 'Krasnodar', 'Saratov', 'Tyumen']

In [33]:
distances_from_moscow = {
    'zlatoust': 1391.263586320666,
    'moscow': 0.0,
    'krasnoyarsk': 3336.039749777034,
    'saint petersburg': 636.1695135872066,
    'sochi': 1361.357154793485,
    'yaroslavl': 251.13992274208584,
    'mytishchi': 19.15256793901058,
    'novorossiysk': 1226.5566120134847,
    'balashikha': 22.57464380465976,
    'pushkino': 32.29150343512499,
    'vladivostok': 6434.301687239597,
    'alexandrov': 99.03461742181614,
    'astrakhan': 1272.5934764474214,
    'reutov': 15.066736856880723,
    'kazan': 721.4395898259049,
    'ulyanovsk': 704.8643503547661,
    'tula': 173.37597779851754,
    'yekaterinburg': 1421.8971980595,
    'rostov-on-don': 959.8830610946284,
    'samara': 856.8315253096929,
    'domodedovo': 36.1846810301295,
    'yoshkar-ola': 645.0905459206801,
    'chelyabinsk': 1498.8592485500628,
    'krasnogorsk': 19.128893627152596,
    'krasnodar': 1114.8111085097096,
    'lipetsk': 353.93468708196895,
    'nakhabino': 29.34753329618301,
    'kyzyl': 3669.160067257817,
    'ryazan': 184.0749867936237,
    'tyumen': 1716.8034849522674,
    'omsk': 2243.230614478407,
    'nizhny novgorod': 403.1271008941555,
    'irkutsk': 3997.3240242086913,
    'mezhdurechensk': 3186.977397404048,
    'stupino': 100.50298403934347,
    'serpukhov': 93.79639415363022,
    'saratov': 727.5243756615845,
    'grozny': 1498.6390016944135,
    'orenburg': 1230.997301228693,
    'surgut': 2143.5097426260527,
    'volgograd': 913.40653000604,
    'engels': 734.1820123219143,
    'fryazino': 35.68825436908673,
    'naberezhnye chelny': 926.5455187130812,
    'khabarovsk': 5955.703005291212,
    'ufa': 1168.153047105715,
    'novosibirsk': 2820.763592183317,
    'kirov': 793.5857281587636,
    'kotelniki': 18.556201509278427,
    'kaluga': 162.98501946171325,
    'vyborg': 758.2625638645857,
    'barnaul': 2943.257935404941,
    'tambov': 419.072146029825,
    'tver': 162.03184502252185,
    'korolyov': 23.240966914958825,
    'kostroma': 302.3271334606748,
    'zheleznodorozhny': 1051.3212963591216,
    'dolgoprudny': 21.451753870279653,
    'kursk': 456.3676328552856,
    'pyatigorsk': 1359.1346795165907,
    'khimki': 19.27787504978214,
    'dubna': 113.336078534706,
    'izhevsk': 970.8632565557036,
    'chita': 4752.38859817778,
    'cherkessk': 1319.672066231367,
    'blagoveshchensk': 5630.258195023883,
    'bryansk': 349.6394555186336,
    'voronezh': 467.015180857099,
    'kolomna': 103.33805649619023,
    'nalchik': 1430.5028996363571,
    'obninsk': 96.82910844583522,
    'belgorod': 577.7869858749181,
    'perm': 1170.262093684163,
    'severodvinsk': 989.3463180226084,
    'gatchina': 616.369780994091,
    'syktyvkar': 1007.3633668496318,
    'naro-fominsk': 68.51450800685191,
    'protvino': 101.15152863427636,
    'kamensk-uralsky': 1506.829417564834,
    'zhukovskiy': 600.4219593564324,
    'tomsk': 2655.7361007309364,
    'stavropol': 1278.9876969538484,
    'ivanteyevka': 31.051927448382365,
    'yegoryevsk': 98.37617987397816,
    'magnitogorsk': 1399.4447582737268,
    'vidnoye': 22.273670822552017,
    'abakan': 3386.4037130516103,
    'kraskovo': 24.96235703886392,
    'safonovo': 286.2924673657379,
    'shlisselburg': 608.2272247268463,
    'cherepovets': 376.65445395984784,
    'arkhangelsk': 992.6837722348444,
    'lyubertsy': 19.131687885364908,
    'vologda': 578.1169534315311,
    'petrozavodsk': 697.9721736505935,
    'oryol': 325.58358692835327,
    'odintsovo': 23.396555594006895,
    'voskresensk': 82.36561436203819,
    'almetyevsk': 934.756507490178,
    'veliky novgorod': 492.2434986334845,
    'kovrov': 240.34824684527445,
    'ramenskoye': 43.322630130479546,
    'berezniki': 1212.7987454012757,
    'shchyolkovo': 30.14037479489325,
    'novocheboksarsk': 617.6614496689281,
    'maykop': 1252.0417154745587,
    'nefteyugansk': 2101.016020281276,
    'tomilino': 587.5370629562508,
    'kaliningrad': 1091.7810203900708,
    'vladimir': 179.35415952478112,
    'velikiye luki': 446.4897914407206,
    'nakhodka': 6512.826751067336,
    'nizhnevartovsk': 2316.101568624056,
    'nevinnomyssk': 1274.9648982402225,
    'klin': 85.25341211585662,
    'uchaly': 1395.0694573085807,
    'novy urengoy': 2357.070596933191,
    'smolensk': 370.11957641629294,
    'dedovsk': 33.44282124708212,
    'sergiyev posad': 70.71201081392101,
    'vsevolozhsk': 629.8608377734972,
    'dmitrov': 66.48470950822428,
    'cheboksary': 602.6003537044186,
    'podolsk': 35.868186236649,
    'pervouralsk': 1380.892115200903,
    'anapa': 1207.7697317191653,
    'lytkarino': 26.171266900562145,
    'kopeysk': 1513.6217762611632,
    'tosno': 582.7586696635991,
    'norilsk': 2890.251515258955,
    'tolyatti': 799.8381096242144,
    'temryuk': 1165.3624515813908,
    'ussuriysk': 6381.347211888504,
    'istra': 50.88951924026617,
    'murmansk': 1491.035365952789,
    'ivanovo': 249.58733461631263,
    'verkhnyaya pyshma': 1418.646108882692,
    'maloyaroslavets': 110.21255938028736,
    'tikhvin': 496.9134080491585,
    'neryungri': 5028.977481470272,
    'kurgan': 1630.4830408964883,
    'novomoskovsk': 198.39912622251103,
    'kemerovo': 2996.19975700017,
    'volokolamsk': 108.56175176242488,
    'elektrostal': 51.84872232603937,
    'essentuki': 1354.1058476615915,
    'kislovodsk': 1365.9041690419383,
    'malakhovka': 26.99082254199157,
    'orekhovo-zuyevo': 85.6729405699771,
    'angarsk': 4178.283677448494,
    'chekhov': 68.44709881944979,
    'severomorsk': 1499.3047699096576,
    'leninsk-kuznetskiy': 3024.4663354351296,
    'satka': 1353.5254445419057,
    'dzerzhinsk': 368.59794417334047,
    'хомутово': 6668.791623791499,
    'penza': 556.0305682853448,
    'biysk': 3068.7132170237933,
    'novokuznetsk': 3127.6022996940287,
    'lobnya': 30.440525152635374,
    'sterlitamak': 1201.8214869231806,
    'naryan-mar': 1546.3123404691437,
    'kamyshin': 819.0343184384951,
    'artyom': 6430.381425066174,
    'yurga': 2914.0593434459674,
    'asbest': 1471.9556267379069,
    'sertolovo': 656.4737697387209,
    'staraya kupavna': 35.69475828769861,
    'volzhskiy': 915.5424228748903,
    'yuzhno-sakhalinsk': 6662.822265097304,
    'dzerzhinsky': 19.755817456584587,
    'krasnoznamensk': 40.16986859789947,
    'ulan-ude': 4432.539381037524,
    'zarechnyy': 606.0887567751221,
    'kostomuksha': 1057.376319264544,
    'povarovo': 50.52734489960697,
    'meleuz': 1226.2104294516412,
    'nazran': 1483.2725769944918,
    'klimovsk': 42.08840658931683,
    'stary oskol': 495.74079214243545,
    'vorkuta': 1891.6302759858595,
    'poltavskaya': 1155.8248883948718,
    'lukhovitsy': 125.0361461564636,
    'beloozyorskiy': 61.073850334859074,
    'elektrougli': 38.19286528690888,
    'chebarkul': 1438.8690266914327,
    'pskov': 611.6967383817062,
    'ukhta': 1249.5980588134028,
    'ruza': 89.5764906198789,
    'nizhny tagil': 1377.8498180844908,
    'yakutsk': 4901.163764820078,
    'khotkovo': 60.72433440755533,
    'elista': 1148.3051053640338,
    'bratsk': 3852.601801304402,
    'noginsk': 52.95761210648052,
    'makhachkala': 1587.5655360528883,
    'vladikavkaz': 1503.498930286972,
    'belovo': 3048.147193273616,
    'frolovo': 780.2333467253039,
    'petropavlovsk-kamchatskiy': 6798.018682170094,
    'borovichi': 370.0201002038788,
    'kratovo': 38.27502643887081,
    'prokopyevsk': 3097.586118228412,
    'seversk': 2878.478697347678,
    'rybinsk': 266.62620678639803,
    'pereslavl-zalessky': 133.65119287257676,
    'taganrog': 953.8988301108136,
    'novouralsk': 1387.9078797741597,
    'gelendzhik': 1245.0497234965765,
    'aramil': 1438.094504062841,
    'neftekamsk': 1037.9314586718494,
    'novotroitsk': 1457.0530345108157,
    'tuapse': 1300.1385167816816,
    'mozhaysk': 103.83921583291148,
    'sarov': 373.8154496798572,
    'kimry': 126.08756924425133,
    'kubinka': 61.218046614594186,
    'zheleznogorsk': 3391.850347906158,
    'saransk': 514.5759444962858,
    'dimitrovgrad': 784.9888159828704,
    'slavyansk-na-kubani': 1167.5855361519575,
    'tikhoretsk': 1114.786471927972,
    'salekhard': 1940.798233727196,
    'ostrov': 596.0147871697434,
    'kommunar': 609.0316563952802,
    'tsivilsk': 617.608810996026,
    'shakhty': 911.990910602426,
    'pechora': 1487.3071484483453,
    'svetogorsk': 784.6154471409538,
    'megion': 2289.965764352699,
    'bataysk': 969.330021045852,
    'kirishi': 530.8309526915012,
    'komsomolsk-on-amur': 6086.29642220532,
    'novocherkassk': 943.5609192173324,
    'chernogolovka': 55.75814822209252,
    'gorki-2': 727.7907922818495,
    'gus-khrustalny': 192.64146200083803,
    'berdsk': 2842.4371674583103,
    'lysva': 1251.2694243399262,
    'korsakov': 6693.46814234493,
    'sovetsk': 996.7695220314324,
    'aprelevka': 41.18219415362496,
    'semibratovo': 209.0654774418882,
    'pavlovsky posad': 64.94947986028654,
    'bor': 406.57781172305175,
    'magadan': 5696.766522816334,
    'kstovo': 413.3420040082879,
    'shatura': 122.7240473979108,
    'losino-petrovsky': 39.0856643165014,
    'yalutorovsk': 1770.7930860972722,
    'ust-ilimsk': 3824.762306477272,
    'pushchino': 101.94688109279312,
    'polysayevo': 3038.5975501321755,
    'prokhladny': 1409.5731328427237,
    'tobolsk': 1866.1259900513485,
    'birobidzhan': 6025.667074308183,
    'lipitsy': 488.33181946214216,
    'volgodonsk': 968.7193376964842,
    'yeysk': 1006.6702432800072,
    'golitsyno': 42.32796602639879,
    'miass': 1420.4111070245615,
    'yelets': 352.953027033769,
    'zvenigorod': 47.72843063332915,
    'selyatino': 47.81307831562977,
    'volkhov': 561.0120810417935,
    'krasnoarmeysk': 52.84859748189735,
    'apatity': 1335.0174311357218,
    'solnechnogorsk': 62.75797256121989,
    'buynaksk': 1591.6091181217846,
    'bolshoy kamen': 6459.6647658063575,
    'noyabrsk': 2258.5967649018207,
    'tynda': 5133.7148203026745,
    'nizhnekamsk': 891.351444472025,
    'usolye-sibirskoye': 4152.668391343766,
    'kingisepp': 673.0970220244146,
    'zelenodolsk': 682.4973369990087,
    'glazov': 950.3194914895024,
    'achinsk': 3216.780147934828,
    'rodniki': 295.0464136688749,
    'shelekhov': 4209.77264099612,
    'kaspiysk': 1601.679936799069,
    'birsk': 1127.659719672664,
    'nadym': 2157.8430358400265,
    'gorno-altaysk': 3143.3301534511943,
    'kolchugino': 3029.386251308493,
    'kushchyovskaya': 1034.055831067238,
    'yuzhnyy': 3482.931087966988,
    'armavir': 1221.3111343868295,
    'mozhga': 909.9695253925483,
    'vyksa': 292.00651478106886,
    'uzlovaya': 200.30676600029304,
    'rostov': 959.8830610946284,
    'kirovsk': 605.2671113612804,
    "ul'yanovka": 192.0411085381513,
    'novoaltaysk': 2950.9705068425405,
    'kamensk-shakhtinsky': 846.2139538784264,
    'ozersk': 1442.8560961441965,
    'elektrogorsk': 74.63716516619932,
    'arzamas': 394.35402804220274,
    'snezhinsk': 1438.9690430594771,
    'borisoglebsk': 569.9580301479704,
    'mirny': 4170.726504147512,
    'vyazniki': 287.308764186897,
    'enem': 1207.821030982559,
    'nyagan': 1734.888769945378,
    'khanty-mansiysk': 1906.792032820939,
    'monino': 37.715349372745926,
    'mineralnye vody': 1342.0944633137724,
    'verkhnyaya salda': 1413.7297090555912,
    'russkiy': 1213.062834379814,
    'revda': 1379.6717524046908,
    'lesnoy gorodok': 28.475504020539947,
    'argun': 1506.098028834171,
    'kyshtym': 1434.2595681626117,
    'salavat': 1210.078586205624,
    'tuchkovo': 74.30089302855156,
    'kumertau': 1226.5856038134475,
    'polevskoy': 1399.151300955637,
    'mikhaylovka': 734.9935677109922,
    'murom': 279.2759175366648,
    'kineshma': 335.5996758437128,
    'kandalaksha': 1299.3304879892798,
    'krasnouralsk': 1384.2334857967037,
    'torzhok': 217.4706587749375,
    'yuzhnouralsk': 1509.4408603749698,
    'shuya': 263.59454702063505,
    'serov': 1426.9672333775843,
    'sosnovy bor': 685.2323134899109,
    'shadrinsk': 1617.147870036188,
    'kotlas': 806.4754172091222,
    'novotitarovskaya': 1173.4741472213989,
    'balakovo': 787.8822091993675,
    '83709': 3202.813333769226,
    'azov': 968.9726699294038,
    'kirovo-chepetsk': 811.8571178014768,
    'rubtsovsk': 2872.4015656846614,
    'kirzhach': 90.62652039638547,
    'anadyr': 6213.777177759523,
    'khasavyurt': 1531.9145979750529,
    'beryozovsky': 1433.634294395171,
    'derbent': 1707.4033646935634,
    'dubovoe': 414.1453826358143,
    'vyazma': 217.54174988855237,
    'pyt-yakh': 2113.4245970564507,
    'semender': 1583.4934150903011,
    'nikolskoye': 598.9997361692872,
    'kashira': 107.13368440394626,
    'kizlyar': 1473.3798545362304,
    'zarinsk': 2997.181989822133,
    'konakovo': 119.1786549464336,
    'kurumoch': 839.7114992062343,
    'udomlya': 285.6169847200254,
    'minusinsk': 3402.637804653251,
    'kropotkin': 1166.2245607801617,
    'aleksandrovsk-sakhalinskiy': 6299.8823011927725,
    'pavlovo': 342.2607377444623,
    'otradny': 924.9037500182482,
    'aksay': 956.2965486068668,
    'balakhna': 381.1579987404556,
    'monchegorsk': 1379.666459717234,
    'kurchatov': 473.2883426264815,
    'orsk': 1466.0377255809008,
    'beloretsk': 1344.6724341902625,
    'vyshny volochyok': 276.9809306440123,
    'plastunovskaya': 1168.6851720045977,
    'nizhnyaya tura': 1375.9166273461458,
    'poronaysk': 6485.758764373887,
    'oktyabrsky': 1018.8184366705572,
    'shakhovskaya': 135.50534368151105,
    'kizilyurt': 1545.4401863233754,
    'chistopol': 821.3604105474976,
    'kansk': 3517.5408678268373,
    'inta': 1658.5575397677376,
    'novoshakhtinsk': 903.6507811060995,
    'kartaly': 1517.4593040784584,
    'kurovskoye': 84.3681700784244,
    'kuznetsk': 652.0791300259867,
    'rossosh': 631.8948563881265,
    'samarskoye': 990.9463636012208,
    'usinsk': 1560.1926768575506,
    'kanevskaya': 1079.414438120802,
    'lyuban': 552.7103608674508,
    'dalnegorsk': 6509.488452820776,
    'urus-martan': 1513.9438781256622,
    'votkinsk': 1018.7575282525304,
    'katav-ivanovsk': 1309.0842038516064,
    'olginka': 1287.7238993616631,
    'yakhroma': 60.64340421625101,
    'novokuybyshevsk': 851.6052629828683,
    'ishimbay': 1214.0401210017262,
    'sosnovoborsk': 3386.3759431879366,
    'gudermes': 1506.983332793841,
    'labytnangi': 1937.057017992013,
    'likino-dulyovo': 84.31241624220513,
    'lodeynoye pole': 604.1151265677914,
    'syzran': 760.0979681269697,
    'belaya kalitva': 870.3865327785855,
    'spassk-dalny': 6362.30839868133,
    'belorechensk': 1232.6961732990508,
    'sarapul': 1008.78232479936,
    'raduzhny': 2358.496974772968,
    'trudovoye': 1090.3401905593514,
    'kinel': 887.7664648811271,
    'korkino': 1505.8826336695265,
    'kiselyovsk': 3086.31280001039,
    'berezovka': 1087.264673590551,
    'goryachevodsky': 1360.7786042757737,
    'michurinsk': 369.2508261957532,
    'bobrovskiy': 322.39759418097685,
    'chegem': 1420.378254627167,
    'yelizovo': 6772.820421777107,
    '53425': 3202.813333769226,
    'тимофеевка': 1006.6834116656815,
    'ust-labinsk': 1180.6818380079214,
    'starokorsunskaya': 1196.1976393224706,
    'tarko-sale': 2388.9780271861964,
    'kachkanar': 1354.197212226372,
    'iskitim': 2858.9099151934147,
    'sibay': 1407.2647547506033,
    'vlasikha': 2933.2726322389262,
    '8756': 10.730176534095364,
    'agoi': 1294.7427424182354,
    'priozersk': 733.1545071632743,
    'bologoye': 321.9803937484453,
    'тарасовка': 1491.2541362483853,
    'beloyarsky': 1836.5384486488076,
    'ust-katav': 1302.724181677159,
    'kanash': 622.9255607421362,
    'dagestanskie ogni': 1698.208239037858,
    'belogorsk': 5631.125637486317,
    'labinsk': 1255.8319936717032,
    'lyskovo': 465.1306258788371,
    'afipsky': 1208.3956944522176,
    'starnikovo': 65.1332863630618,
    'juravskaia': 1139.1135698400817,
    'korenovsk': 1151.523891415271,
    'znamenskiy': 1194.8142115320263,
    'slavgorod': 2644.610086488067,
    'kungur': 1200.4961224183248,
    'gadzhiyevo': 1519.9109169685858,
    'shchyokino': 194.4698174383533,
    'nogliki': 6268.259965863986,
    '39404': 3202.813333769226,
    'lenina': 1199.0498830456509,
    'gaiduk': 1219.513851211696,
    '9992': 982.845979460969,
    'marks': 751.1509601828144,
    'kugesi': 606.0520067073529,
    'solikamsk': 1216.98913220201,
    'bryukhovetskaya': 1111.082927529371,
    'slavyanka': 6428.684709591151,
    '24130': 3202.813333769226,
    'menzelinsk': 970.9489283044122,
    'vysokaya gora': 724.5834162625722,
    'vanino': 6362.894335239535,
    'malysheva': 1467.3592923926346,
    'korzhevskiy': 1171.954752073123,
    'troitsk': 1538.140668806304,
    'nizhnebakanskaya': 1210.9986252672918,
    'plast': 1482.6858068116453,
    'gorodishche': 591.2765666147258,
    'osinovo': 736.6860205116334,
    'zavidovo': 353.2224718264869,
    'zarechny': 1465.4555186281548,
    'novaya adygeya': 1196.6866369650947,
    'kholmsk': 6621.858284986929,
    'asha': 1245.1907526999598,
    '14076': 3202.813333769226,
    'petrovskoye': 385.1673685384485,
    'chernyakhovsk': 1012.0952920550218,
    'salsk': 1067.2210501046166,
    'argayash': 1458.8443906207276,
    '13403': 1152.7191232199125,
    'bavly': 1006.55370599998,
    'beslan': 1482.7229051085862}


def get_distance_from_moscow(city: str) -> float:
    """Возвращает расстояние от города `city` до Москвы в километрах. 
    Для неизвестных и зарубежных городов возвращает -1.
    """

    return distances_from_moscow.get(str(city).lower(), -1.0)

In [34]:
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_valid_preprocessed = preprocessor.transform(X_valid)

print(f'X_train.shape = {X_train_preprocessed.shape}')

X_train.shape = (1460042, 55)


### Выбор метрик  

Будем использовать в качестве основной метрики - `roc_auc`. Но также взглянем и на другие метрики: `accuracy`, `precision`, `recall`, `f1`.  


In [35]:
def find_best_threshold(
    y_true: pd.Series, 
    y_proba: pd.Series, 
    metriс_name: str = 'roc_auc', 
    iterations: int = 250, 
    learning_rate: float = 0.05
) -> float:
    """Находит лучший порог перевода вероятностей `y_proba` 
    в принадлежность к классу 1.
    """
    
    # Получение функции метрики, которую оптимизируем
    metrics = {'roc_auc': roc_auc_score, 'f1': f1_score, 
               'precision': precision_score, 'recall': recall_score}
    metric_function = metrics.get(metriс_name, accuracy_score)

    # Получение метрики
    def get_metric(threshold: float) -> float:
        prediction = (y_proba > threshold).astype(int)
        return metric_function(y_true, prediction)

    direction = -1
    shift = 0.25

    best_threshold = 0.5
    best_metric = get_metric(best_threshold)

    # На каждой итерации
    for i in range(iterations):

        # Меняем порог
        threshold = best_threshold + direction * shift
        shift *= (1 - learning_rate)
        metric = get_metric(threshold)

        # И проверяем, улучшилась ли метрика
        if metric > best_metric: 
            best_threshold = threshold
            best_metric = metric
        else: 
            direction *= -1
            
    return best_threshold

In [36]:
def print_metrics(
    model: BaseEstimator, 
    X: pd.DataFrame, 
    y: pd.Series, 
    threshold: Union[float, None] = None, 
    show_roc_curve: bool = False
) -> None:

    # Получим предсказания, если возможно в виде вероятностей
    try: 
        probas = model.predict_proba(X)[:, 1]
    except AttributeError:
        prediction = model.predict(X)
        threshold = None
        probas = None
    else:
        threshold = threshold or find_best_threshold(y, probas, 'roc_auc')
        prediction = (probas > threshold).astype(int)

    # Распечатаем порог перевода вероятностей в классы
    if threshold is None:
        print("Порог перевода вероятностей в классы: не используется")
    else:
        print(f"Порог перевода вероятностей в классы: {threshold}")
        print(f"{roc_auc_score(y, probas)} - roc_auc на вероятностях")

    # Распечатаем однострочные метрики
    print()
    print(f"{roc_auc_score(y, prediction):0.8f} - roc_auc")
    print(f"{accuracy_score(y, prediction):0.8f} - accuracy")
    print(f"{precision_score(y, prediction):0.8f} - precision")
    print(f"{recall_score(y, prediction):0.8f} - recall")
    print(f"{f1_score(y, prediction):0.8f} - f1")

    # Распечатаем матрицу ошибок
    conf_mat = confusion_matrix(y, prediction)
    classes = model.classes_
    n_classes = len(classes)
    print()
    print("|".join(f"{i:^10}" for i in ["prediction"] + list(classes)))
    print(f"{'true label':^10}" + ("|" + " " * 10) * n_classes)
    print("-" * ((n_classes * 10) + n_classes + 10))
    for i in range(n_classes):
        print("|".join(f"{j:>10}" for j in [classes[i]] + list(conf_mat[i])))

    # Отобразим ROC-кривую
    if show_roc_curve:
        print()
        plt.figure(figsize=(7, 4))
        if probas is not None:
            plt.plot(*roc_curve(y_test, probas)[:2], 
                     c='r', label='on probability')
        plt.plot(*roc_curve(y_test, prediction)[:2], c='b', label='on class')
        plt.plot([0, 1], [0, 1], c='y', label='random', linestyle='dashed')
        plt.title('Receiver operating characteristic')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend()
        plt.show()

### Базовая модель  

Так как целевая переменная распределена не равномерно, то в качестве бейзлайна можно выбрать стратегию, которая для каждой сессии предсказывает значение 0.  


In [37]:
# Обучим базовую модель 
baseline = DummyClassifier(strategy='constant', constant=0)
baseline.fit(X_train_preprocessed, y_train)

# И получим её метрики
print_metrics(baseline, X_valid_preprocessed, y_valid, 0.5)

Порог перевода вероятностей в классы: 0.5
0.5 - roc_auc на вероятностях

0.50000000 - roc_auc
0.97295000 - accuracy
0.00000000 - precision
0.00000000 - recall
0.00000000 - f1

prediction|   0.0    |   1.0    
true label|          |          
--------------------------------
       0.0|    194590|         0
       1.0|      5410|         0


### Выбор модели  

Обучим несколько моделей, которые нам известны и выберем лучшую, которую уже будем оптимизировать.

#### Логистическая регрессия

In [38]:
logreg = LogisticRegression(random_state=RANDOM_SEED)

In [39]:
%%time
logreg.fit(X_train_preprocessed, y_train);

CPU times: total: 42.7 s
Wall time: 11.1 s


In [40]:
print_metrics(logreg, X_valid_preprocessed, y_valid)

Порог перевода вероятностей в классы: 0.5
0.6683122284030721 - roc_auc на вероятностях

0.50000000 - roc_auc
0.97295000 - accuracy
0.00000000 - precision
0.00000000 - recall
0.00000000 - f1

prediction|   0.0    |   1.0    
true label|          |          
--------------------------------
       0.0|    194590|         0
       1.0|      5410|         0


In [41]:
# Логистическая регрессия проходит roc_auc

#### Метод опорных векторов

In [42]:
svc = LinearSVC(class_weight='balanced')


In [43]:
%%time
svc.fit(X_train_preprocessed, y_train);

CPU times: total: 9min 35s
Wall time: 9min 37s


In [44]:
print_metrics(svc, X_valid_preprocessed, y_valid)

Порог перевода вероятностей в классы: не используется

0.62028562 - roc_auc
0.61587000 - accuracy
0.04324746 - precision
0.62495379 - recall
0.08089678 - f1

prediction|   0.0    |   1.0    
true label|          |          
--------------------------------
       0.0|    119793|     74797
       1.0|      2029|      3381


In [45]:
# Метод опорных векторов не проходит roc_auc

#### Многослойный персептрон

In [46]:
mlp = MLPClassifier((32,), random_state=RANDOM_SEED)

In [47]:
%%time
mlp.fit(X_train_preprocessed, y_train);

CPU times: total: 5min 6s
Wall time: 1min 20s


In [48]:
print_metrics(mlp, X_valid_preprocessed, y_valid)

Порог перевода вероятностей в классы: 0.5
0.6982130825521674 - roc_auc на вероятностях

0.50000000 - roc_auc
0.97295000 - accuracy
0.00000000 - precision
0.00000000 - recall
0.00000000 - f1

prediction|   0.0    |   1.0    
true label|          |          
--------------------------------
       0.0|    194590|         0
       1.0|      5410|         0


In [49]:
# Многослойный персептрон проходит roc_auc

#### Случайный лес

In [50]:
forest = RandomForestClassifier(random_state=RANDOM_SEED)

In [51]:
%%time
forest.fit(X_train_preprocessed, y_train);

CPU times: total: 5min 16s
Wall time: 5min 17s


In [52]:
print_metrics(forest, X_valid_preprocessed, y_valid)

Порог перевода вероятностей в классы: 0.010218123817113232
0.6253519433580381 - roc_auc на вероятностях

0.59594638 - roc_auc
0.59998000 - accuracy
0.03952047 - precision
0.59168207 - recall
0.07409208 - f1

prediction|   0.0    |   1.0    
true label|          |          
--------------------------------
       0.0|    116795|     77795
       1.0|      2209|      3201


In [53]:
# Случайный лес не проходит roc_auc

### Оптимизация модели  

Лучшей моделью является MLPClassifier по следующим причинам:
+ Один из лучших показателей `roc_auc`.
+ Быстрое обучение. 
+ Модель интерпретируема, то есть можно получить показатели важности признаков.

Оптимизация модели и конвейера по подготовке данных проводится с помощью байесовской оптимизации. При разных гиперпараметрах модель обучается на тренировочных данных, а оценивается на валидационных.

In [54]:
    # Создадим конвейер с заданными гиперпараметрами
    model = Pipeline(steps=[
        # Создание дополнительных признаков
        ('indexer', FunctionTransformer(set_index)), 
        ('imputer', FunctionTransformer(fill_missings)), 
        ('engineer', FunctionTransformer(create_features)), 
        ('dropper', DropFeatures(['client_id', 'visit_date', 'visit_time', 
                                  'device_screen_resolution'])), 
        # Преобразования численных переменных
        ('normalization', YeoJohnsonTransformer()), 
        ('outlier_remover', Winsorizer()), 
        ('scaler', SklearnTransformerWrapper(StandardScaler())), 
        # Преобразования категориальных признаков
        ('rare_encoder', RareLabelEncoder(replace_with='rare')),
        ('onehot_encoder', OneHotEncoder(drop_last_binary=True)), 
        ('bool_converter', FunctionTransformer(converse_types)), 
        # Удаление дубликатов и коррелируемых признаков
        ('constant_dropper', DropConstantFeatures()), 
        ('duplicated_dropper', DropDuplicateFeatures()), 
        ('correlated_dropper', DropCorrelatedFeatures()), 
        # Лучшая модель с оптимизированными гиперпараметрами
        ('model', MLPClassifier( 
            random_state=RANDOM_SEED, batch_size=256, verbose=True, early_stopping=True))])
    
    # Обучим и оценим модель
    model.fit(X_train, y_train)
    prediction = model.predict_proba(X_valid)[:, 1]
    return roc_auc_score(y_valid, prediction)

Iteration 1, loss = 0.12050511
Validation score: 0.972953
Iteration 2, loss = 0.11857216
Validation score: 0.972953
Iteration 3, loss = 0.11811853
Validation score: 0.972953
Iteration 4, loss = 0.11786109
Validation score: 0.972953
Iteration 5, loss = 0.11764496
Validation score: 0.972953
Iteration 6, loss = 0.11748358
Validation score: 0.972953
Iteration 7, loss = 0.11735609
Validation score: 0.972953
Iteration 8, loss = 0.11727137
Validation score: 0.972953
Iteration 9, loss = 0.11717053
Validation score: 0.972946
Iteration 10, loss = 0.11710842
Validation score: 0.972946
Iteration 11, loss = 0.11700463
Validation score: 0.972953
Iteration 12, loss = 0.11688827
Validation score: 0.972953
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


SyntaxError: 'return' outside function (3330711562.py, line 28)

In [ ]:
X_train

In [ ]:
y

## Оценка модели

In [ ]:
final_pipeline = Pipeline(steps=[

    # Создание дополнительных признаков и
    # Приведение датафрейма к удобному виду 
    ('indexer', FunctionTransformer(set_index)), 
    ('imputer', FunctionTransformer(fill_missings)), 
    ('engineer', FunctionTransformer(create_features)), 
    ('dropper', DropFeatures(['client_id', 'visit_date', 'visit_time', 
                              'device_screen_resolution'])), 

    # Преобразования численных переменных
    ('normalization', YeoJohnsonTransformer()), 
    ('outlier_remover', Winsorizer()), 
    ('scaler', SklearnTransformerWrapper(StandardScaler())), 

    # Преобразования категориальных признаков
    ('rare_encoder', RareLabelEncoder(tol=0.047319, replace_with='rare')),
    ('onehot_encoder', OneHotEncoder(drop_last_binary=True)), 
    ('bool_converter', FunctionTransformer(converse_types)), 

    # Удаление дубликатов и коррелируемых признаков
    ('constant_dropper', DropConstantFeatures(tol=0.95579)), 
    ('duplicated_dropper', DropDuplicateFeatures()), 
    ('correlated_dropper', DropCorrelatedFeatures(threshold=0.8856)), 

    # Лучшая модель с оптимизированными гиперпараметрами
    ('model', MLPClassifier(random_state=RANDOM_SEED, batch_size=256, verbose=True, early_stopping=True)),])

### Метрики модели

Для оценки метрик модели обучим её на объектах тренировочной и валидационной выборках и сделаем предсказания на тестовых данных. 

Целевая метрика `roc-auc=0.6998` (для пресказанных классов) выбранной модели превосходит 0.65, а значит, цель работы выполнена.


In [ ]:
final_pipeline.fit(X, y);

In [ ]:
test_proba = final_pipeline.predict_proba(X_test)[:, 1]
best_threshold = find_best_threshold(y_test, test_proba)
test_prediction = (test_proba > best_threshold).astype(int)

print(f'Лучший порог перевода вероятностей в класс: {best_threshold}')

In [ ]:
print(f'Метрики лучшей модели на обучающей выборке:')
print_metrics(final_pipeline, X, y, best_threshold)

In [ ]:
print(f'Метрики лучшей модели на тестовой выборке:')
print_metrics(
    final_pipeline, X_test, y_test, best_threshold, show_roc_curve=True)

### Обучение на всех данных

Для анализа обработки данных и важности признаков разобъём финальный конвейер на препроцессор и модель и обучим их на всех данных. А перед сохранением модели объединим обратно.

In [ ]:
final_model = final_pipeline.named_steps['model']
final_preprocessor = final_pipeline.set_params(model=None)

In [ ]:
sessions_preprocessed = final_preprocessor.fit_transform(sessions)

In [ ]:
final_model.fit(sessions_preprocessed, target);

### Анализ обработки данных

В итоге всех преобразований получается 62 признака, при том, что ещё 16 признаков были удалены из-за корреляций и т.п..

Датасет имеет 176667 дубликатов, но как показали эксперименты, удаление дубликатов из тренировочной выборки ведёт к небольшому ухудшению метрик. 

Признаков, коррелируемых с целевой переменной, нет.

In [ ]:
sessions_preprocessed.shape

In [ ]:
sessions_preprocessed.info()

In [ ]:
print('Количество дубликатов:', sessions_preprocessed.duplicated().sum())

In [ ]:
print('Корреляция с целевой переменной:')
correlation = pd.concat([sessions_preprocessed, target], axis=1).corr()
correlation['target'].sort_values(ascending=False, key=abs).head(5)

### Важность признаков

Самыми важными признаками после преобразования оказались численные переменные: день (месяца и недели), час, минута (скорее всего именно нулевая минута часа) и номер посещения, размеры экрана и расстояние до Москвы.

После этих признаков важность остальных сильно падает. Возможно такова особенность работы с категориальными признаками (особенно с большим количеством неравномерно распределенных уникальных значений).

Самыми важными исходными признаками окзаались: размеры экрана, дата, время и номер посещения, город пользователя, а также признаки с дополнительными данными (как индикатор органического трафика). 

In [ ]:
print('Признаки, удалённые во время feature selection\n')
all = 0

for step in ('constant_dropper', 'duplicated_dropper', 'correlated_dropper'):
    print(step + ':')
    for column in final_preprocessor.named_steps[step].features_to_drop_:
        print(f'\t{column}')
        all += 1

print(f'Всего удалено: {all}')

### Сохранение модели

In [ ]:
# Объединим препроцессор и модель обратно

final_pipeline = final_preprocessor.set_params(model=final_model)

In [ ]:
# Добавим метаданные для модели

metadata = {
    'name': 'SberAutopodpiska: target event prediction', 
    'descripton': ('Модель по предсказанию совершения пользователем одного из '
                   'целевых действий "Заказать звонок" или "Оставить заявку" '
                   'на сайте сервиса СберАвтоподписка.'), 
    'model_type': final_model.__class__.__name__, 
    'version': 1.0, 
    'training_datetime': datetime.now(), 
    'author': 'Igor Putsenko', 
    'threshold': best_threshold, 
    'metrics': {
        'roc_auc': roc_auc_score(y_test, test_proba), 
        'roc_auc_by_class': roc_auc_score(y_test, test_prediction),
        'accuracy': accuracy_score(y_test, test_prediction), 
        'precision': precision_score(y_test, test_prediction), 
        'recall': recall_score(y_test, test_prediction), 
        'f1': f1_score(y_test, test_prediction),
    }
}

final_pipeline.metadata = metadata

In [ ]:
# Сохраним модель

models_folder.mkdir(exist_ok=True)
filename = f'model_{datetime.now():%Y%m%d%H%M%S}.pkl'

with open(models_folder / filename, 'wb') as file:
    dill.dump(final_pipeline, file)

## Выводы

Для преобразования входных данных, со структурой как в файле `ga_sessions.csv`, в удобный для предсказания вид понадобилось четыре этапа:
1. Заполнение пропусков и генерация признаков. В том числе добавление новых данных, как-то органический трафик или расстояние до Москвы (ох и не просто это было).
2. Преобразование численных переменных: нормализация и удаление выбросов.
3. Преобразование категориальных признаков. Основная сложность с ними была в многообразии редких уникальных значений. В итоге только самые популярные значения были закодированы методом one-hot.
4. Удаление дублирующих и коррелируемых признаков. Признаки могут коррелировать до 0.95, но именно с таким порогом финальная модель даёт лучший результат.

В итоге в качестве лучшей модели был выбран `MLPClassifier` по следующим причинам: 
+ Один из лучших показателей `roc_auc`.
+ Быстрое обучение. 
+ Может предсказывать вероятность класса.

Качество модели по метрике `roc-auc` составляет **0.698**, на тестовой выборке 0.68, правда, на финальном обучении модели она несколько снизилась, и составила **0.698** .Тем не менее, переобучения нет и цель проекта выполнена - `roc-auc` > 0.65.